# Importing the Libraries

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import re
import nltk
import sklearn
import catboost

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Importing the dataset

In [2]:
index_col=["HelpfulnessNumerator", "HelpfulnessDenominator", "Score", "Summary", "Text"]
dataset = pd.read_csv("Reviews.csv", usecols=index_col)
dataset.head()

,HelpfulnessNumerator,HelpfulnessDenominator,Score,Summary,Text
0,3,3,5,Just like the original one in Saudi Arabia.,''''' '''' '' '''''''' ..<br />Just like the o...
1,0,0,5,Arrived Fast,-- - - - Item arrived in perfect condition and...
2,2,3,5,GUMBO! BEER! Are you ready for some football!,-- -- OK - Getcher beer! ---- Getcher Minute R...
3,0,0,5,Cat loves these!,- Been buying these for my cat for about a yea...
4,0,0,5,Pure bliss in a pod --,"-- better yet, 96 pods of caffeine bliss. Thi..."


In [3]:
X            = dataset.iloc[:,4:].values   #Taking out review text
votesHelpful = dataset.iloc[:,0].values  #Helpfulness for summary
votesTotal   = dataset.iloc[:,1].values

## Creating Helfulnesss index

In [4]:
print(len(votesHelpful))
y=[]

393568


In [5]:
for i in range(len(votesHelpful)):
    y.append(votesHelpful[i]/votesTotal[i])
    if(y[i]>=0.8):
        y[i] = 1
    else:
        y[i] = 0

In [6]:
print(len(y))

393568


In [7]:
print(X)

[["''''' '''' '' '''''''' ..<br />Just like the original one in Saudi Arabia.<br />Very recommended for Saudis who live in The United States."]
 ['-- - - - Item arrived in perfect condition and very quickly. Would definitely buy from them again. - - - ---']
 ["-- -- OK - Getcher beer! ---- Getcher Minute Rice and Hotty-Sauce! ---- Nuke yer Campbell's Gumbo! ---- Put it all together!---- ------Open a cool one!---Enjoy the game! ----"]
 ...
 ["Zuke's turkey n' taterz mini bakes are a great purchase.  Not only is the price fair but the product is easy to dispense as well as to fill up kongs with or other toys designed for treats.  As a training tool Zukes mini bakes are perfect.  They are just the right size to reward my dogs with as I work on training them.  I've had no issues with indigestion, gas, or any other stomach problems with Zuke's products and will be ordering them again and again.  They are all natural, which goes a long way for me.  No need to give our dogs chemicals or prese

# Cleaning the Dataset

### Listing Contractions

In [8]:
contractions={
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd've": "how did have",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "might have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "shall'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "will't've": "will not have",
    "would've": "would have",
    "would't": "would not",
    "would't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you have all",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have",
}

### Declaring Cleaning function:(contraction removal, html removed using regex, removed  stopwords by NLTK)

In [9]:
def clean_text(text, remove_stopwords = True):
    text = text.lower()
    if True:    # remove contractions
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    text = re.sub(r'https?:\/\/.*[\r\n]*', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', ' ', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    cleanr = re.compile('<.*?>')
    text = re.sub(cleanr, ' ', text)
    ps = PorterStemmer()
    if remove_stopwords:    # remove stopwords
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [ps.stem(word) for word in text if not word in set(stops)] #using Porter Stemmer
        text = " ".join(text)
    return text

### Applying clean_texts on individual line

In [10]:
for i in range(len(X)):
    for j in range(len(X[i])):
        X[i][j] = clean_text(X[i][j])
print(X)

[['like origin one saudi arabia recommend saudi live unit state']
 ['item arriv perfect condit quickli would definit buy']
 ['ok getcher beer getcher minut rice hotti sauc nuke yer campbel gumbo put togeth open cool one enjoy game']
 ...
 ['zuke turkey n taterz mini bake great purchas price fair product easi dispens well fill kong toy design treat train tool zuke mini bake perfect right size reward dog work train issu indigest ga stomach problem zuke product order natur goe long way need give dog chemic preserv give tri pleas']
 ['zuma hairless chines crest pup finicki eater would rather starv eat someth like food eat cesar gourmet filet sauc ok find dog treat like husband brought home cesar grill chicken softi dog treat love treat look smell like dog treat thought way zuma would eat hundr dollar spent numer bog bakeri year half spent shop place found cesar grill chicken softi dog treat thrill reliev']
 ['zzzzzzz high hope coffe realli disappoint say better buy folger ad extra scoop fi

In [11]:
print(y)

[1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 

## Making A Corpus

In [12]:
corpus = X.flatten()

In [13]:
print(X)

[['like origin one saudi arabia recommend saudi live unit state']
 ['item arriv perfect condit quickli would definit buy']
 ['ok getcher beer getcher minut rice hotti sauc nuke yer campbel gumbo put togeth open cool one enjoy game']
 ...
 ['zuke turkey n taterz mini bake great purchas price fair product easi dispens well fill kong toy design treat train tool zuke mini bake perfect right size reward dog work train issu indigest ga stomach problem zuke product order natur goe long way need give dog chemic preserv give tri pleas']
 ['zuma hairless chines crest pup finicki eater would rather starv eat someth like food eat cesar gourmet filet sauc ok find dog treat like husband brought home cesar grill chicken softi dog treat love treat look smell like dog treat thought way zuma would eat hundr dollar spent numer bog bakeri year half spent shop place found cesar grill chicken softi dog treat thrill reliev']
 ['zzzzzzz high hope coffe realli disappoint say better buy folger ad extra scoop fi

In [14]:
print(corpus)

['like origin one saudi arabia recommend saudi live unit state'
 'item arriv perfect condit quickli would definit buy'
 'ok getcher beer getcher minut rice hotti sauc nuke yer campbel gumbo put togeth open cool one enjoy game'
 ...
 'zuke turkey n taterz mini bake great purchas price fair product easi dispens well fill kong toy design treat train tool zuke mini bake perfect right size reward dog work train issu indigest ga stomach problem zuke product order natur goe long way need give dog chemic preserv give tri pleas'
 'zuma hairless chines crest pup finicki eater would rather starv eat someth like food eat cesar gourmet filet sauc ok find dog treat like husband brought home cesar grill chicken softi dog treat love treat look smell like dog treat thought way zuma would eat hundr dollar spent numer bog bakeri year half spent shop place found cesar grill chicken softi dog treat thrill reliev'
 'zzzzzzz high hope coffe realli disappoint say better buy folger ad extra scoop filter buy us

In [15]:
len(corpus)

393568

## Tokenizing

In [16]:
tokenizer = Tokenizer(num_words = 2000, oov_token="<OOV>")

In [17]:
tokenizer.fit_on_texts(corpus)

In [18]:
word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'like': 2, 'tast': 3, 'flavor': 4, 'good': 5, 'product': 6, 'use': 7, 'one': 8, 'love': 9, 'great': 10, 'tri': 11, 'coffe': 12, 'would': 13, 'tea': 14, 'get': 15, 'make': 16, 'food': 17, 'buy': 18, 'time': 19, 'realli': 20, 'eat': 21, 'order': 22, 'much': 23, 'amazon': 24, 'price': 25, 'also': 26, 'littl': 27, 'find': 28, 'drink': 29, 'dog': 30, 'bag': 31, 'well': 32, 'even': 33, 'best': 34, 'store': 35, 'cup': 36, 'better': 37, 'go': 38, 'chocol': 39, 'box': 40, 'mix': 41, 'sugar': 42, 'water': 43, 'day': 44, 'sweet': 45, 'look': 46, 'first': 47, 'year': 48, 'packag': 49, '2': 50, 'want': 51, 'recommend': 52, 'could': 53, 'give': 54, 'brand': 55, 'purchas': 56, 'found': 57, 'think': 58, 'made': 59, 'way': 60, 'bought': 61, 'work': 62, 'treat': 63, 'free': 64, 'say': 65, 'enjoy': 66, '1': 67, 'bit': 68, 'know': 69, 'thing': 70, 'need': 71, 'review': 72, 'cannot': 73, 'differ': 74, 'nice': 75, 'two': 76, 'sinc': 77, '3': 78, 'add': 79, 'delici': 80, 'lot': 81, 'cat': 82, 's

## Inputs to text Sequences

In [19]:
input_sequences = []
input_sequences=tokenizer.texts_to_sequences(corpus)
#input_sequences = tokenizer.texts_to_matrix(corpus)
#for line in corpus:
#    token_list = tokenizer.texts_to_sequences([line])
#    for i in range(1, len(token_list)):
#        n_gram_sequence = token_list[:i+1]
#        input_sequences.append(n_gram_sequence)
# pad sequences 


In [20]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=200, padding='post'))

In [21]:
print(input_sequences)

[[   2  368    8 ...    0    0    0]
 [ 165  189  124 ...    0    0    0]
 [ 436    1 1095 ...    0    0    0]
 ...
 [   1 1043 1006 ...    0    0    0]
 [   1    1 1286 ...    0    0    0]
 [   1  148  244 ...    0    0    0]]


In [22]:
print(len(y))

393568


In [23]:
print(len(input_sequences))

393568


# Splitting the Dataset into Train Set and Test Set


In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(input_sequences, y, test_size = 0.2, random_state = 0)

# Trying different ML Models

## Decision Tree

### Training On Training Data

In [25]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

### Testing on Test Data

In [26]:
y_pred = classifier.predict(X_test)
y_test = np.array(y_test)
print(np.concatenate((y_test.reshape(len(y_test), 1), y_pred.reshape(len(y_pred), 1)), 1))

[[1 0]
 [0 1]
 [0 0]
 ...
 [0 1]
 [0 0]
 [0 0]]


### Confusion Matrix

In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[31461 18330]
 [17693 11230]]


0.5423558706202201

## XGBoost

### Training

In [28]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

### Testing

In [29]:
y_pred = classifier.predict(X_test)
y_test = np.array(y_test)
print(np.concatenate((y_test.reshape(len(y_test), 1), y_pred.reshape(len(y_pred), 1)), 1))

[[1 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Confusion Matrix

In [30]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[47451  2340]
 [26739  2184]]


0.6305739766750514

## CatBoost

### Training

In [31]:
from catboost import CatBoostClassifier
classifier = CatBoostClassifier()
classifier.fit(X_train, y_train)

Learning rate set to 0.120123
0:	learn: 0.6832530	total: 318ms	remaining: 5m 18s
1:	learn: 0.6755680	total: 481ms	remaining: 3m 59s
2:	learn: 0.6696940	total: 695ms	remaining: 3m 51s
3:	learn: 0.6651030	total: 843ms	remaining: 3m 29s
4:	learn: 0.6615651	total: 980ms	remaining: 3m 15s
5:	learn: 0.6587684	total: 1.11s	remaining: 3m 4s
6:	learn: 0.6566017	total: 1.23s	remaining: 2m 54s
7:	learn: 0.6549501	total: 1.38s	remaining: 2m 50s
8:	learn: 0.6536590	total: 1.54s	remaining: 2m 50s
9:	learn: 0.6526555	total: 1.67s	remaining: 2m 45s
10:	learn: 0.6518409	total: 1.84s	remaining: 2m 45s
11:	learn: 0.6512218	total: 2.01s	remaining: 2m 45s
12:	learn: 0.6507123	total: 2.17s	remaining: 2m 44s
13:	learn: 0.6503149	total: 2.33s	remaining: 2m 44s
14:	learn: 0.6500010	total: 2.46s	remaining: 2m 41s
15:	learn: 0.6497472	total: 2.62s	remaining: 2m 41s
16:	learn: 0.6495489	total: 2.76s	remaining: 2m 39s
17:	learn: 0.6493661	total: 2.92s	remaining: 2m 39s
18:	learn: 0.6492294	total: 3.08s	remaining: 

### Testing

In [32]:
y_pred = classifier.predict(X_test)
y_test = np.array(y_test)
print(np.concatenate((y_test.reshape(len(y_test), 1), y_pred.reshape(len(y_pred), 1)), 1))

[[1 0]
 [0 0]
 [0 1]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Confusion Matrix

In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[47980  1811]
 [27133  1790]]


0.6322890464212211

In [78]:
new_review = "If you buy this from Amazon, you'll be disappointed at the shipping.  My box of 12 came in one cardboard box, with the 12 container-boxes loosely in a plastic bag.  One was smashed, so leaking into the bag, two were dented, and one was coming open from"
new_review = clean_text(new_review)
new_corpus = [new_review]
new_X_test = tokenizer.texts_to_sequences(new_corpus)
prediction_sequence = np.array(pad_sequences(new_X_test, maxlen=200, padding='post'))
new_y_pred = classifier.predict(prediction_sequence)
print(new_y_pred)

[0]


In [76]:
print(new_X_test)

[[18, 24, 191, 93, 40, 310, 214, 8, 1134, 40, 310, 121, 40, 713, 380, 31, 8, 1, 1125, 31, 76, 916, 8, 85, 147]]


In [77]:
print(new_review)

buy amazon disappoint ship box 12 came one cardboard box 12 contain box loos plastic bag one smash leak bag two dent one come open
